In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
import os

In [5]:
path = 'H:/'
folders_to_remove = ['misc','Seagate','System Volume Information', '$RECYCLE.BIN']
output = [dI for dI in os.listdir(path) if os.path.isdir(os.path.join(path,dI))]
output_cleaned = [x for x in output if x not in folders_to_remove]
output_cleaned

['02A',
 '02B',
 '02C',
 '02D',
 '02E',
 '06A',
 '06B',
 '06C',
 '06D',
 '06E',
 '10A',
 '10B',
 '10C',
 '10D',
 '10E',
 '14A',
 '14B',
 '14C',
 '14D',
 '18A',
 '18B',
 '18C']

In [11]:
singleCellData = pd.read_csv('C:/Users/chris.langseth/Downloads/scRNAseq_new.csv')

In [12]:
for i, slide_tag in enumerate(output_cleaned): 
    print(slide_tag)
    decoded = pd.read_csv('I:/'+ slide_tag +'/preprocessing/decoded_2134.csv')
    decoded = decoded[decoded.name != 'NNNN']
    step = 125
    to_bin = lambda x: np.floor(x / step) * step
    decoded["pos1_bin"] = decoded['pos1'].map(to_bin)
    decoded["pos2_bin"] = decoded['pos2'].map(to_bin)
    groups = decoded.groupby(["pos1_bin", "pos2_bin"])
    bin_coordinate = list(groups.groups.keys())
    bin_id = list(range(1, len(groups.groups.keys())+1))
    coordinate_to_id = dict(zip(bin_coordinate, bin_id))
    decoded['bin_coordinate'] = list(zip(decoded.pos1_bin, decoded.pos2_bin))
    decoded['bin_id'] = decoded['bin_coordinate'].map(coordinate_to_id)
    group_binID = decoded.groupby(by = 'bin_id')
    frame_with_counts = (decoded['bin_id'].value_counts()).to_frame().reset_index()
    frame_with_counts_sorted = frame_with_counts.sort_values(by='index')
    decoded_sorted = decoded.sort_values(by = 'bin_id')
    frame_with_counts_sorted = frame_with_counts_sorted.reset_index(drop=True)
    decoded_sorted = decoded_sorted.reset_index(drop=True)
    index_list1 = frame_with_counts_sorted['index']
    bin_id_list1 = frame_with_counts_sorted['bin_id']
    add_count_dict = dict(zip(index_list1,bin_id_list1))
    
    decoded_sorted['count'] = frame_with_counts_sorted['bin_id']
    decoded_sorted['count'] = decoded_sorted['bin_id'].map(add_count_dict)
    decoded_sorted.loc[decoded_sorted['count'] < 4, "bin_id"] = int(0)
    decoded_sorted = decoded_sorted.sort_values(by='bin_id')
    notZero = decoded_sorted[decoded_sorted['bin_id'] != 0]
    bin_id_new = list(range(1, len(list(np.unique(notZero.bin_id)))+1))
    old_bind_id = (list(np.unique(notZero.bin_id)))
    old_to_new_binID = dict(zip(old_bind_id,bin_id_new))
    old_to_new_binID[0] = 0
    decoded_sorted['bin_id_new'] = decoded_sorted['bin_id'].map(old_to_new_binID)
    decoded_sorted.loc[decoded_sorted['bin_id_new'] == 0, "pos1_bin"] = np.NaN
    decoded_sorted.loc[decoded_sorted['bin_id_new'] == 0, "pos2_bin"] = np.NaN
    
    spots = pd.DataFrame()
    spots['x_global'] = decoded_sorted['pos1']
    spots['y_global'] = decoded_sorted['pos2']
    spots['fov_id'] = 0
    spots['label'] = decoded_sorted['bin_id_new']
    spots['target'] = decoded_sorted['name']
    spots['x_cell'] = decoded_sorted['pos1_bin']
    spots['y_cell'] = decoded_sorted['pos2_bin']
    
    genesOfInterest = list(singleCellData.GeneName)
    spots2 = spots[spots.target.isin(genesOfInterest)]
    
    label_list = list(np.unique(spots.label))
    label_id_list = list(np.unique(spots.label))
    del label_id_list[0]
    
    cells = pd.DataFrame()
    cells['cell_id'] = np.array(label_id_list)-1
    cells['label'] = label_id_list
    cells['fov_id'] = 0
    cells['area'] = 15625
    
    label_list_not_unique = list(spots.label)
    x_cell_list = list(spots.x_cell)
    y_cell_list = list(spots.y_cell)
    
    label_to_xcell = dict(zip(label_list_not_unique, x_cell_list))
    label_to_ycell = dict(zip(label_list_not_unique, y_cell_list))
    
    cells['x'] = cells['label'].map(label_to_xcell)
    cells['y'] = cells['label'].map(label_to_ycell)
    
    spots2.to_csv('I:/cells_spots_files/spots_bins'+slide_tag+'.csv')
    cells.to_csv('I:/cells_spots_files/cells_bins'+slide_tag+'.csv')

02A
02B
02C
02D
02E
06A
06B
06C
06D
06E
10A
10B
10C
10D
10E
14A
14B
14C
14D
18A
18B
18C
